#  LangChain의 개념과 주요 컴포넌트 이해

---

## LangChain이란 

- **LangChain**은 LLM 기반 애플리케이션 개발을 위한 프레임워크

- **Chain**은 작업을 순차적으로 실행하는 파이프라인 구조를 제공

- **Agent**는 자율적 의사결정이 가능한 실행 단위


## LangChain 컴포넌트 

- **언어 처리 기능**은 LLM/ChatModel이 중심이 되며, Prompt와 Memory로 대화를 관리

- **문서 처리와 검색**은 Document Loader, Text Splitter, Embedding, Vectorstore가 담당

- **모듈성**이 핵심 특징으로, 독립적인 컴포넌트들을 조합해 RAG와 같은 복잡한 시스템을 구현 가능 

---

# 환경 설정 및 준비

In [6]:
# 라이브러리 설치
# uv add ipykernel python-dotenv langchain langchain-openai langchain_google_genai

In [7]:
# .env 파일 설정
# OPENAI_API_KEY=your_openai_api_key
# GOOGLE_API_KEY=your_google_api_key

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv
import os
load_dotenv(override=True) # override=True

# 키가 잘 로드되었는지 확인 (앞 7자리만 출력)
api_key = os.getenv("LANGSMITH_API_KEY")
if api_key:
    print(f"API Key 로드 성공: {api_key[:7]}...")
else:
    print("API Key를 찾을 수 없습니다. .env 파일을 확인해주세요.")

API Key 로드 성공: lsv2_pt...


In [2]:
from langsmith import Client
client = Client()

for p in client.list_projects():
    print(f"[{p.name}]")


[default]
[001_chatbot]


# 1. 모델 (Models)
- LLM, ChatModel 등으로 구분
- OpenAI, Anthropic, Google 등 다양한 모델을 지원
- 텍스트 생성, 대화, 요약 등의 작업을 수행

In [ ]:
# langsmith api key가 잘 안 먹어서 chatgpt가 디버깅?용으로 제공해준 코드
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='What is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='255fc416-5d87-45d0-aa1e-10b908b1bf11'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 142, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5trwGWnsr3yl1hTpXaBZlSOCf26k', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2e07-898d-7452-86a8-075b457033de-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_Li2xXDvffny4Iu1ZuaofQSL1', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 142,

In [4]:
from langchain_openai import ChatOpenAI

# OpenAI 모델을 사용하여 대화 생성
model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.3)

# 모델에 메시지를 보내고 응답을 받기
response = model.invoke("안녕하세요! 초록피아노토끼입니다. 오늘은 초록 봄 공주로 보냈어요")

In [5]:
# 응답 객체(AIMessage 클래스): 메시지(content)와 메타데이터(response_metadata 등)를 포함
response

AIMessage(content='안녕하세요! 초록피아노토끼님, 오늘은 초록 봄 공주로 보내셨다니 정말 특별한 하루였겠어요. 어떤 이야기나 소식이 있으시면 언제든 들려주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 31, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_70fc5e01a5', 'id': 'chatcmpl-D5ts6Kq9LoOrpOLtfWXLIJMSFI5kC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2e07-b35e-7690-a1c6-2e87cd8462ff-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 31, 'output_tokens': 47, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
# 응답 객체의 메시지 내용 출력
print("답변: ", response.content)

답변:  안녕하세요! 초록피아노토끼님, 오늘은 초록 봄 공주로 보내셨다니 정말 특별한 하루였겠어요. 어떤 이야기나 소식이 있으시면 언제든 들려주세요!


In [7]:
# 응답 객체의 메타데이터 출력
print("메타데이터: ", response.response_metadata)

메타데이터:  {'token_usage': {'completion_tokens': 47, 'prompt_tokens': 31, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_70fc5e01a5', 'id': 'chatcmpl-D5ts6Kq9LoOrpOLtfWXLIJMSFI5kC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore

# 1. LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. 문서
documents = [
    Document(page_content="LangSmith는 LLM 애플리케이션의 observability 도구다."),
    Document(page_content="LangChain은 LLM 체인을 만들기 위한 프레임워크다."),
    Document(page_content="RAG는 검색 결과를 프롬프트에 포함하는 기법이다."),
]

# 3. In-memory vector store
embeddings = OpenAIEmbeddings()
vectorstore = InMemoryVectorStore.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# 4. Prompt
prompt = ChatPromptTemplate.from_template(
    """
    context:
    {context}

    question:
    {question}
    """
)

# 5. Chain
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 6. 실행
print(chain.invoke("LangSmith는 어떤 도구야?"))


LangSmith는 LLM 애플리케이션의 observability 도구입니다.


### [실습] 

- Google Gemini 모델을 사용하여 텍스트 생성하고 응답 객체를 확인해보세요.

- 참고: https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [ ]:
# 여기에 코드를 작성하세요. 
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview", # pro, gemini-3-flash-preview, gemini-2.5-pro
    temperature=1.0,  # Gemini 3.0+ defaults to 1.0
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = model.invoke(messages)
ai_msg

AIMessage(content=[{'type': 'text', 'text': "J'adore la programmation.", 'extras': {'signature': 'EuwFCukFAb4+9vvC72V1SRiAE3t0yZdmNqp67LKX+hl/9cRKalJnS/t9b3QNbL/DnqmQGwUx9eEK3ixdA+g/fm71FFuAlJIv0qCqIuAHyR7Vvdu28L/tZNNl4EVgF8DHzvTPVYEMHDFw8hO7sxzD7OkPvGk0df5SdaiPuj1MZ6HAdHeHP5h1V1kUea2EeJbXIWJgcn9us3sFc25V8gRs7h5cZODZ9fBbRpSP8guwNx2vdlzxfJORDwaOWNiM8K7dBSqlSY+16+Yt+7SGyTXls2iqIEZtS6oVbszQpeJkENowaR9mlgyQ5PswWwqq9CPJrqmHB9uSGDiyLLU7kdiVj74dyXxXwndwoxlBgmLUazolOrnYkJN0m7TivB9eHdQhmkebKcKRmCmHKUOttsQ5ljNogGzQpVbyfmAMORrV6bv/DYle7RGSjo6UYXjREJYmkkfsl8qt0uYIvcaXbNBqX7IRCi329TFPejioEjW7vNeTD6RwGDoHgskdO5uH5vDP4CV9N3HmDVy0e6TvELWjY1ho7mNcZhoDIKRHN+jAIdRNVNCozVIXKoV8WJZbTEy9sKvMRZHohNMTbicurJbPw5Ey5cJpuZdAXRqfS8CL7hBvleQNfWPf+fMDC2rHuBugHQx1IJd003vLAPxK3EcBI6JCpLLH5dnP+ZOsvymYNuI3ByPG6o32lZhkPD33gW+rwysoyTF8jTvGx2q9M28xzawudiW9m8q3web5AeLOhdHqtxRNWaaPHsKHkMc6WFXvpCliZW8BETToOX0QcR8iK3ITKP2r3CyTeLrqNmIs3ITg5o4KgKcvZ47deWzvVaQon4lq8mFYj+okqnbmBO5TIuMBEQlR0Vx3IqAU+CJBW8mrbiXB/2jDZOO+YL/dFoutmYdpeL

# 2. 메시지 (Messages)
- Chat Model에서 사용할 수 있는 통합된 메시지 형식을 제공
- 각 모델 제공자의 특정 메시지 형식을 신경 쓰지 않고도 다양한 채팅 모델을 활용 가능

`1. HumanMessage`
- 사용자 역할에 해당 (user, human 등)
- 사용자의 입력을 처리

In [13]:
from langchain_core.messages import HumanMessage

# 사용자 메시지 생성
human_message = HumanMessage(content="Glory를 한국어로 번역해주세요.")

# 번역 요청 및 응답 받기
response = model.invoke([human_message])  # 메시지 리스트로 전달

# 답변 출력
print("답변: ", response.content)

답변:  [{'type': 'text', 'text': "'Glory'는 문맥에 따라 다음과 같이 번역될 수 있습니다.\n\n### 1. 가장 일반적인 의미\n*   **영광 (Yeong-gwang):** 명예나 찬사, 영예로운 영광을 뜻할 때 가장 많이 쓰입니다.\n    *   예: Glory to the winner. (승자에게 **영광**을.)\n    *   예: Glory to God. (하나님께 **영광**을.)\n\n### 2. 찬란한 아름다움이나 장엄함\n*   **찬란함 / 영광 (Chan-ran-ham):** 눈부시게 아름다운 상태나 장관을 의미할 때 쓰입니다.\n    *   예: The glory of the sunset. (석양의 **찬란함/아름다움**)\n\n### 3. 명예나 영예\n*   **영예 (Yeong-ye):** 사회적인 명성이나 영광스러운 명예를 강조할 때 쓰입니다.\n    *   예: He brought glory to his country. (그는 나라에 **영예**를 안겨주었다.)\n\n### 4. 전성기\n*   **전성기 / 황금기 (Jeon-seong-gi):** 가장 화려했던 시절을 의미할 때 쓰입니다.\n    *   예: In his former glory. (그의 과거 **전성기** 시절에.)\n\n---\n\n**참고:**\n넷플릭스 드라마 제목인 **<더 글로리(The Glory)>**처럼 고유명사나 제목으로 쓰일 때는 한국어에서도 그대로 외래어인 **'글로리'**라고 표기하기도 합니다. 이 드라마의 맥락에서는 주인공이 되찾고자 하는 '영광' 혹은 '찬란한 삶'을 의미합니다.", 'extras': {'signature': 'EvcJCvQJAb4+9vuITpXZHCpdZ5ieeXtIw58gLPCbc/GOUE9uxL/lyejKWOdm7kMGQONvPI4bQs0e5k6UPEsBAfLLXOWFgkkd2J/eCr2PSwOmC9n3MRgELjm0jd4h0P/xRiwmSVEPp1HHTm0LMOYqYtLquy2

In [14]:
response

AIMessage(content=[{'type': 'text', 'text': "'Glory'는 문맥에 따라 다음과 같이 번역될 수 있습니다.\n\n### 1. 가장 일반적인 의미\n*   **영광 (Yeong-gwang):** 명예나 찬사, 영예로운 영광을 뜻할 때 가장 많이 쓰입니다.\n    *   예: Glory to the winner. (승자에게 **영광**을.)\n    *   예: Glory to God. (하나님께 **영광**을.)\n\n### 2. 찬란한 아름다움이나 장엄함\n*   **찬란함 / 영광 (Chan-ran-ham):** 눈부시게 아름다운 상태나 장관을 의미할 때 쓰입니다.\n    *   예: The glory of the sunset. (석양의 **찬란함/아름다움**)\n\n### 3. 명예나 영예\n*   **영예 (Yeong-ye):** 사회적인 명성이나 영광스러운 명예를 강조할 때 쓰입니다.\n    *   예: He brought glory to his country. (그는 나라에 **영예**를 안겨주었다.)\n\n### 4. 전성기\n*   **전성기 / 황금기 (Jeon-seong-gi):** 가장 화려했던 시절을 의미할 때 쓰입니다.\n    *   예: In his former glory. (그의 과거 **전성기** 시절에.)\n\n---\n\n**참고:**\n넷플릭스 드라마 제목인 **<더 글로리(The Glory)>**처럼 고유명사나 제목으로 쓰일 때는 한국어에서도 그대로 외래어인 **'글로리'**라고 표기하기도 합니다. 이 드라마의 맥락에서는 주인공이 되찾고자 하는 '영광' 혹은 '찬란한 삶'을 의미합니다.", 'extras': {'signature': 'EvcJCvQJAb4+9vuITpXZHCpdZ5ieeXtIw58gLPCbc/GOUE9uxL/lyejKWOdm7kMGQONvPI4bQs0e5k6UPEsBAfLLXOWFgkkd2J/eCr2PSwOmC9n3MRgELjm0jd4h0P/xRiwmSVEPp1HHTm

In [15]:
# 문자열을 입력하면, 자동으로 HumanMessage로 변환하여 요청
model.invoke("Glory를 한국어로 번역해주세요.")

AIMessage(content=[{'type': 'text', 'text': '\'Glory\'는 문맥에 따라 여러 가지 한국어로 번역될 수 있습니다. 가장 대표적인 의미들은 다음과 같습니다.\n\n### 1. 가장 일반적인 의미\n*   **영광 (Yeong-gwang):** 가장 보편적인 번역입니다. 명예롭고 빛나는 영예를 의미합니다.\n    *   예: Glory to God (하나님께 영광을)\n    *   예: Morning glory (나팔꽃 - 식물 이름이지만 \'아침의 영광\'이라는 뜻이 있음)\n\n### 2. 찬란함이나 빛\n*   **광채 (Gwang-chae):** 눈부시게 빛나는 빛을 의미합니다.\n*   **찬란함 (Challan-ham):** 화려하고 눈부신 상태를 의미합니다.\n\n### 3. 명예와 승리\n*   **영예 (Yeong-ye):** 영광스러운 명예를 뜻합니다.\n*   **부귀영화 (Bugui-yeonghwa):** 재산이 많고 지위가 높아 귀하게 되며 몸이 귀하게 되어 이름이 세상에 빛남을 의미할 때 \'Glory\'를 쓰기도 합니다.\n\n### 4. 고유 명사 (드라마 등)\n*   **더 글로리 (The Glory):** 넷플릭스 드라마 제목처럼 고유 명사로 쓰일 때는 한국어에서도 외래어 표기 그대로 \'글로리\'라고 부릅니다.\n\n---\n\n**상황별 예시:**\n*   "In all his **glory**" -> "그의 모든 **영광** 속에서" 또는 "가장 **찬란한** 모습으로"\n*   "Glory of victory" -> "승리의 **영광**"\n*   "The sun shone in all its **glory**" -> "태양이 **찬란하게** 비치고 있었다"\n\n어떤 문맥에서 사용하시려는지 알려주시면 더 정확한 번역을 도와드릴 수 있습니다.', 'extras': {'signature': 'EqALCp0LAb4+9vvzfcbtsHqLcUmY4CCMdmaJWaOpYc28wpmpuuz

`2. AIMessage`
- AI 모델의 응답을 표현


In [16]:
# AI 모델의 응답 객체를 출력 
response

AIMessage(content=[{'type': 'text', 'text': "'Glory'는 문맥에 따라 다음과 같이 번역될 수 있습니다.\n\n### 1. 가장 일반적인 의미\n*   **영광 (Yeong-gwang):** 명예나 찬사, 영예로운 영광을 뜻할 때 가장 많이 쓰입니다.\n    *   예: Glory to the winner. (승자에게 **영광**을.)\n    *   예: Glory to God. (하나님께 **영광**을.)\n\n### 2. 찬란한 아름다움이나 장엄함\n*   **찬란함 / 영광 (Chan-ran-ham):** 눈부시게 아름다운 상태나 장관을 의미할 때 쓰입니다.\n    *   예: The glory of the sunset. (석양의 **찬란함/아름다움**)\n\n### 3. 명예나 영예\n*   **영예 (Yeong-ye):** 사회적인 명성이나 영광스러운 명예를 강조할 때 쓰입니다.\n    *   예: He brought glory to his country. (그는 나라에 **영예**를 안겨주었다.)\n\n### 4. 전성기\n*   **전성기 / 황금기 (Jeon-seong-gi):** 가장 화려했던 시절을 의미할 때 쓰입니다.\n    *   예: In his former glory. (그의 과거 **전성기** 시절에.)\n\n---\n\n**참고:**\n넷플릭스 드라마 제목인 **<더 글로리(The Glory)>**처럼 고유명사나 제목으로 쓰일 때는 한국어에서도 그대로 외래어인 **'글로리'**라고 표기하기도 합니다. 이 드라마의 맥락에서는 주인공이 되찾고자 하는 '영광' 혹은 '찬란한 삶'을 의미합니다.", 'extras': {'signature': 'EvcJCvQJAb4+9vuITpXZHCpdZ5ieeXtIw58gLPCbc/GOUE9uxL/lyejKWOdm7kMGQONvPI4bQs0e5k6UPEsBAfLLXOWFgkkd2J/eCr2PSwOmC9n3MRgELjm0jd4h0P/xRiwmSVEPp1HHTm

In [17]:
# 응답 객체의 자료형 확인
type(response)

langchain_core.messages.ai.AIMessage

In [18]:
# 모델 응답 텍스트 부분을 출력
response.content

[{'type': 'text',
  'text': "'Glory'는 문맥에 따라 다음과 같이 번역될 수 있습니다.\n\n### 1. 가장 일반적인 의미\n*   **영광 (Yeong-gwang):** 명예나 찬사, 영예로운 영광을 뜻할 때 가장 많이 쓰입니다.\n    *   예: Glory to the winner. (승자에게 **영광**을.)\n    *   예: Glory to God. (하나님께 **영광**을.)\n\n### 2. 찬란한 아름다움이나 장엄함\n*   **찬란함 / 영광 (Chan-ran-ham):** 눈부시게 아름다운 상태나 장관을 의미할 때 쓰입니다.\n    *   예: The glory of the sunset. (석양의 **찬란함/아름다움**)\n\n### 3. 명예나 영예\n*   **영예 (Yeong-ye):** 사회적인 명성이나 영광스러운 명예를 강조할 때 쓰입니다.\n    *   예: He brought glory to his country. (그는 나라에 **영예**를 안겨주었다.)\n\n### 4. 전성기\n*   **전성기 / 황금기 (Jeon-seong-gi):** 가장 화려했던 시절을 의미할 때 쓰입니다.\n    *   예: In his former glory. (그의 과거 **전성기** 시절에.)\n\n---\n\n**참고:**\n넷플릭스 드라마 제목인 **<더 글로리(The Glory)>**처럼 고유명사나 제목으로 쓰일 때는 한국어에서도 그대로 외래어인 **'글로리'**라고 표기하기도 합니다. 이 드라마의 맥락에서는 주인공이 되찾고자 하는 '영광' 혹은 '찬란한 삶'을 의미합니다.",
  'extras': {'signature': 'EvcJCvQJAb4+9vuITpXZHCpdZ5ieeXtIw58gLPCbc/GOUE9uxL/lyejKWOdm7kMGQONvPI4bQs0e5k6UPEsBAfLLXOWFgkkd2J/eCr2PSwOmC9n3MRgELjm0jd4h0P/xRiwmSVEPp1HHTm0LMOYqYtLquy20

In [19]:
# 토큰 사용량 출력
response.usage_metadata

{'input_tokens': 10,
 'output_tokens': 783,
 'total_tokens': 793,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 390}}

`3. SystemMessage`
- 시스템 역할에 해당 (system, developer 등)
- AI 모델의 동작과 제약사항을 정의하는데 사용


In [20]:
from langchain_core.messages import SystemMessage 

# 시스템 메시지 생성
system_msg = SystemMessage(content="당신은 영어를 한국어로 번역하는 AI 어시스턴트입니다.")

# 메시지 객체 확인
system_msg

SystemMessage(content='당신은 영어를 한국어로 번역하는 AI 어시스턴트입니다.', additional_kwargs={}, response_metadata={})

In [ ]:
# 번역 요청(HumanMessage)과 시스템 메시지(SystemMessage)를 함께 사용
human_message = HumanMessage(content="Glory")
messages = [system_msg, human_message]

# 모델에 메시지를 보내고 응답 받기
response = model.invoke(messages)

# 답변 출력
print("답변: ", response.content)

답변:  [{'type': 'text', 'text': "'Glory'는 한국어로 다음과 같이 번역할 수 있습니다.\n\n1. **영광** (가장 보편적인 번역)\n2. **영예** (명예로운 영광)\n3. **찬란함/장관** (눈부시게 아름다운 모습)\n\n**예시:**\n* Glory to God: 하나님께 **영광**을\n* Morning glory: 나팔꽃 (또는 아침의 **장관**)\n* The glory of victory: 승리의 **영광**", 'extras': {'signature': 'EswICskIAb4+9vt0PRs/YdJT27dQ5gmNf+iCvGNBfg4fWIloCKgJq4odJXcYrsWK4v9cBWJl9GGVieYdtFB2b+2GiLeoctYP5ZxnIAdKVcYwkXYmHADjEuOqtr5r+MWuYq1vba+WczPFZ4jVHVbDP0ZIQ2ebS/8oQ+YOJZmBfR76058ITqPbi1nS2S/zwu59zlJJCoiRlj/4/zDbM4MuZm97kjeIwm3uGRqpktlb+GTGXdjzeGQjZi5A9jcAWaVBgjkJPUYbR25yEShFV6XlKzOXfEplmox4f2ZC/g4A2RXU7Jp9Trhvb89ncUmoyhKTINBe32AsYE3B5iJWcI7ciZtH5yGmO5//keubw6M6Umvj0JaGslDUVAZMdR6JGZApDEJNl4hcDOcfEdINw0J1NddfXk8Ny6hFEwOqHxI6KB8K4WuXmM8G8I+VLhg+sTtg9pNRiF+eY0zW7BlOLwVErHHJUm+bkgUIH033FrUtgMwUDPovsJL9xcIv+Z7kxKCLCe7tzzK/SLIWS77KK4bqdt2eershrLoaNKdz1177KGxNHWDkITXMcxelxNP6rYPn1sVc2TPQyPU8VX0HIeP4UMypOVtBOaqG+bvI8+ueIPGtoxvYKn3l27l/EV5g2I13GwWQpA2dch1Y9Dj93/4JqsGtTBt2wOsDn636oenrrhUEuzGPt5f5+ysAHUxAGcGvyNRHlHWQxTOD

### [실습]

- Google Gemini 모델을 사용하여, 챗 메시지 목록을 기반으로 텍스트 생성하고 응답 객체를 확인해보세요.

- 참고: https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [ ]:
# 여기에 코드를 작성하세요. 

# 3. 프롬프트 템플릿 (Prompt Template)
- 프롬프트 템플릿을 통해 일관된 입력 형식을 제공
    1. 사용자의 입력과 파라미터를 언어 모델이 이해할 수 있는 형태로 변환하는 도구
    2. 언어 모델에게 전달할 지시문을 만드는 틀
- 변수를 포함한 동적 프롬프트 생성이 가능
    1. 모든 템플릿은 딕셔너리 형태의 입력을 받아서 처리
    2. 출력은 PromptValue 형태로 반환되며, 이는 문자열이나 메시지 리스트로 변환 가능

`1. 문자열 프롬프트 템플릿 (String PromptTemplate)`
- 가장 기본적인 형태
- 단일 문자열을 형식화하는데 사용

In [22]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성 
# "{topic}에 대한 이야기를 해줘"라는 템플릿을 사용하여
# topic이라는 변수를 포함하는 프롬프트를 생성
template = PromptTemplate.from_template("{topic}에 대한 이야기를 해줘")

# 템플릿 사용
# "고양이"라는 주제를 사용하여 프롬프트 생성
# invoke 메서드를 통해 템플릿에 값을 전달
prompt = template.invoke({"topic": "강아지"})

# 템플릿 출력
prompt

StringPromptValue(text='강아지에 대한 이야기를 해줘')

`2. 채팅 프롬프트 템플릿 (ChatPromptTemplate)`
- 여러 메시지를 포함하는 대화형 템플릿을 만들 때 사용

In [23]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅 템플릿 생성
# 여기서는 시스템 메시지와 사용자 메시지를 포함하여 정의
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    ("user", "{subject}에 대해 설명해주세요")
])

# 템플릿 사용
prompt = template.invoke({"subject": "인공지능"})

# 출력
prompt

ChatPromptValue(messages=[SystemMessage(content='당신은 도움이 되는 비서입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='인공지능에 대해 설명해주세요', additional_kwargs={}, response_metadata={})])

In [24]:
prompt.messages

[SystemMessage(content='당신은 도움이 되는 비서입니다', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='인공지능에 대해 설명해주세요', additional_kwargs={}, response_metadata={})]

`3. 메시지 플레이스홀더 (MessagesPlaceholder)`
- 기존 메시지 목록을 템플릿의 특정 위치에 삽입할 때 사용
e.g. 과거 채팅 기록, agent의 작업 일지, few-shot에서 모델이 참고할 수 있는 예시

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 메시지 플레이스홀더가 있는 템플릿
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    MessagesPlaceholder("chat_history")   # 채팅 기록을 플레이스홀더로 사용 (예: 이전 대화 내용) -> 이 위치에 메시지 목록을 추가할 수 있음
])

# 템플릿 사용
# runtime에 아래와 같은 메시지 목록을 전달
prompt = template.invoke({
    "chat_history": [
        HumanMessage(content="안녕하세요! 제 이름은 스티브입니다."),
        AIMessage(content="안녕하세요! 무엇을 도와드릴까요?"),
        HumanMessage(content="제 이름을 기억하나요?")
        ]
})

# 출력
prompt.messages

[SystemMessage(content='당신은 도움이 되는 비서입니다', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕하세요! 제 이름은 스티브입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='제 이름을 기억하나요?', additional_kwargs={}, response_metadata={})]

In [26]:
model.invoke(prompt)

AIMessage(content=[{'type': 'text', 'text': '네, 당연히 기억하고 있습니다. **스티브** 님 맞으시죠? \n\n무엇을 도와드릴까요?', 'extras': {'signature': 'Eu4ECusEAb4+9vvyXbYhjkXZ/DJT49I7ST+X2EuRxN3qjagXU/p/TNQihSJ3ZdtULKFFObqvmtdNraVKPepWPGl+e/SY3iGKgMb/hBQ+5iNXgSeh9leXnuhymvhqXh55P2K9fAdafU1AzDPWqbyOwfJiEWu0K2t1xSdpkcrJSzLpKiINTxHBMAMFKYkwazYYUCR/jri/KEn90Or1kaBQmt4NnWP8Vcy4zgx+iDC9JtEhff2clcjSqwwGccxXKILrxS2ako914RN3SN1j8EggzxhV1e34IbRNRxHPckyaVVwx2G3QLGLQNtDI93cnVK16s7H9lXlQIgVKMYA7+mBCVYFiqmmsijX758TW33AdaMxd9NtGCRsYWYGckopjMc73zXGEQ4/rrzRS0owKFPPZOpnV5I2bwVNBItVv789m/dmiQ7rPfwZUfPpKcZS3C9WVaM055G78GMRwfWD7Nz+zUSTXDccK+UGePJI22HvqRUbmLzJp1yDVyEp1n/0D+W7bzFqej/AFHtVJWhMeLmqTrp2weD4E9KeKVu55DHx8E/0F8WEXfJ3JSZz/peY5932ErLlmAAjTck//qglLragDsF2EV0h7e/0BhXbZjXbga7S0CO7U3vXmYWpc6NKe9DcZy7Qe/iVBqGjqyEmRPDTgtjsjNxtgDMqk6yaFi2Q290KTcu9XWyqOC6BCWhe24U/33rfLfzwdq5OXi4YlDRihjt4xMLlQiWphVy1h4BcOhDLj6PCptKeqQ8dB4ztV9wnHWq+9vl9RHMV0X5IC8GI0SOrrajf3PoO+fxBq/PNi3t28wHGB9al/wa95m4sfSm53ZA=='}}], additional_kwargs={}, response_metad

### [실습] 

- 프롬프트 템플릿을 사용하여, 입력받은 텍스트를 요약하는 템플릿을 작성하고, Google Gemini 모델(2.5 flash 등)을 사용하여 요약 결과를 확인해보세요.

In [ ]:
# 여기에 코드를 작성하세요. 
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI bot. You specialise in summarizing long documents."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

prompt = template.invoke(
    {
        "name": "Bobbie",
        "user_input": "What is your name? And what do you do?",
    }
)

prompt.messages
# Output:
# ChatPromptValue(
#    messages=[
#        SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
#        HumanMessage(content='Hello, how are you doing?'),
#        AIMessage(content="I'm doing well, thanks!"),
#        HumanMessage(content='What is your name?')
#    ]
# )

[SystemMessage(content='You are a senior backend engineer in finance domain. Your name is Bobbie.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='What is your name? And what do you do?', additional_kwargs={}, response_metadata={})]

# 4. 출력 파서 (Output Parser)
1. **역할과 기능**
    - 모델의 텍스트 출력을 구조화된 데이터로 변환
    - 채팅 모델과 LLM의 출력을 정규화
    - 다운스트림 작업을 위한 데이터 형식 변환

2. **사용 시 고려사항**
    - OpenAI function calling과 같은 기능이 있는 경우, 해당 기능을 우선 사용

`(1) StrOutputParser`

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 기본적인 문자열 파서 사용
parser = StrOutputParser()

# 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요")

# 모델 정의
model = ChatOpenAI(model='gpt-4.1-mini')

# LCEL 체인 구성
chain = prompt | model | parser

# 체인 실행
result = chain.invoke({"city": "무이네"}) # 서울

# 결과 출력
print(result)

무이네(Muine)는 베트남 남부에 위치한 해변 마을로, 특히 아름다운 해변과 독특한 자연 경관으로 유명한 관광지입니다. 무이네의 주요 특징은 다음과 같습니다:

1. **아름다운 해변**  
   무이네는 길고 고운 모래 해변이 펼쳐져 있어 해수욕과 해변 휴양에 적합합니다. 맑은 바다와 잔잔한 파도 덕분에 수영, 일광욕, 카이트서핑 등의 해양 스포츠를 즐기기에 좋습니다.

2. **사막 같은 붉은 모래 언덕**  
   무이네 주변에는 붉은색 모래언덕과 하얀 모래언덕이 있어 '베트남의 작은 사막'으로 불립니다. 이 모래 언덕들은 일출과 일몰 시 매우 아름다운 풍경을 자아내며, 스케이트 보드와 같은 활동도 인기가 많습니다.

3. **낚시 마을**  
   무이네는 원래 작은 어촌 마을로서 신선한 해산물을 즐길 수 있으며, 어부들의 전통적인 삶을 엿볼 수 있는 곳입니다.

4. **기후**  
   무이네는 건조하고 따뜻한 기후를 가지고 있어 연중 내내 관광하기 좋은 편입니다. 특히 11월부터 4월까지가 방문하기에 가장 쾌적한 시기로 꼽힙니다.

5. **관광 인프라**  
   최근 들어 리조트, 호텔, 레스토랑 등이 많이 개발되어 편리한 관광 환경을 제공하고 있습니다.

이처럼 무이네는 자연 경관과 해양 활동, 신선한 해산물, 독특한 모래 언덕 풍경 등 다양한 매력을 가진 베트남의 인기 여행지입니다.


`(2) 구조화된 출력 (with_structured_output 메소드)`

In [40]:
from pydantic import BaseModel, Field
from typing import List

# Pydantic 클래스로 출력 구조를 정의 -> json 스키마로 모델에게 전달됨
class NewsInfo(BaseModel):
    """뉴스 기사에 대한 정보"""
    title: str = Field(description="기사 제목")
    content: str = Field(description="기사 내용 (원문 그대로 추출)")
    press: str = Field(description="언론사")
    author: str = Field(description="작성자(예: 길동,홍 = '홍길동'이라는 이름의 given name, surname 순서로 기재)")
    published_date: List[str] = Field(description="작성일(예: 2023년 1월 1일)")

In [42]:
# 뉴스 기사 예제 데이터
news_article = """
"복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발
입력2025.06.20. 오후 12:00  수정2025.06.20. 오후 1:39 

[서울=뉴시스]윤현성 기자 = 정부가 공공분야에 도입할 초거대 인공지능(AI) 기반 국민 편의 서비스 본격 개발에 나선다. 세금 납부 시 생성형 AI 챗봇을 통해 어려운 세무 용어 등에 대한 설명·상담을 듣거나, 대량의 민원업무도 생성형 AI 기반 분석으로 빠르게 답변·응대하는 등의 형태다.

과학기술정보통신부와 디지털플랫폼정부위원회는 2025년도 '초거대 AI 서비스 개발 지원' 사업을 본격 추진하기 위해 수행기업 공모를 실시한다고 20일 밝혔다. 이 사업은 공공분야에 초거대 AI를 도입·확산하고 이를 통해 행정 효율화, 대국민 서비스 혁신, 사회현안 해결이 가능한 서비스 개발을 목표로 추진된다.

올해는 다양한 공공분야에서 초거대 AI 기술을 통해 국민이 체감할 수 있고 실질적인 변화를 가져올 수 있는 과제를 중점적으로 발굴했다. 중앙부처·지자체·공공기관을 대상으로 1~2월에 과제 공모를 추진했으며 총 5개 과제가 선정됐다.

국민권익위원회의 '생성형 AI 기반 국민소통·민원분석 체계 구축'은 국민소통시스템에 생성형 AI 기반 민원분석 체계를 도입해 민원처리 행정 효율화와 답변품질을 향상시킨다. 이를 통해 대량의 민원업무를 신속·효율적으로 대응해 민원업무의 효율성을 증대하고 국민의 신뢰도 향상에도 기여할 것으로 기대된다.

국세청의 '생성형 AI 기반 국세 상담 지원 서비스'는 납세자가 홈택스 이용 시 전자신고 관련한 문의사항을 즉시 해소할 수 있는 실시간 상담 서비스를 제공한다. 홈택스에 상담전용 AI챗봇을 도입해 전화 상담 시 발생하는 장시간 대기 문제를 해결하고, 어려운 세무 용어 등으로 인한 불편사항을 개선할 예정이다.

산업통상자원부의 '해외인증 공공특화 AI 에이전트 서비스'는 모바일 플랫폼, 소셜네트워크서비스 등 사용자 친화적인 모바일 기반 해외인증 특화 AI 에이전트 서비스를 제공한다. 중소기업이 겪는 수출 관련 애로사항인 해외 인증과 관련된 정보와 질의 응답을 AI기반으로 제공하여 손쉽게 확인할 수 있을 것으로 기대된다.

국민건강보험공단의 '에이전틱 AI기반 전국민 맞춤형 민원 상담 서비스'는 국민 생활과 편익에 직결되는 건강보험 민원 상담업무에 AI를 도입해 24시간 개인 맞춤형 민원 상담 서비스를 구현한다. 기존의 전화 상담 방식의 대기 시간 문제 등을 해소하고, 고객센터 집중 상담을 분산시켜 업무의 효율성을 향상시키는데 기여할 것으로 전망이다.

한국지역정보개발원의 '지방재정 지능화 서비스'는 e호조+, 지방재정365 등 지방재정서비스에 생성형 AI를 접목시켜 대국민, 공무원 등 각자의 요구에 부합하는 융복합 재정정보서비스 환경을 제공하고자 한다. 이 서비스가 도입되면 지방정부의 사회현안 해결을 위한 정책 수립의 적시성 향상 및 전문성 확보, 지자체 정보 접근성 강화로 대국민의 사회 참여 기회를 확대할 것으로 예상된다.

이번 사업은 19일 국민권익위원회와 국세청 과제의 민간 전문기업 조달 공고를 시작으로, 5개 과제별 서비스 개발지원 사업이 순차적으로 입찰공고될 예정이다.

김경만 과기정통부 인공지능기반정책관은 "선정된 과제에 대해 민·관 협력을 기반으로 행정 현장의 변화와 국민이 체감할 수 있는 성과가 창출·확산될 수 있도록 적극 지원할 계획"이라며 "개발된 서비스는 공공분야에서 행정업무의 효율성을 높이고 대국민 서비스 품질을 높이는 데 실질적으로 기여할 수 있도록 민간에서도 많은 관심을 가져주실 것을 부탁드린다"고 말했다.

이승현 디플정위 인공지능·플랫폼혁신국장은 "이번 사업은 노동, 복지, 민원 등 다양한 공공 분야에 AI를 도입·활용하는데 중요한 역할을 하고 있다"며 "올해도 AI를 활용해 사회문제를 해결하고, 대국민 서비스를 혁신적으로 개선할 수 있는 서비스가 개발되길 기대한다"고 전했다．

윤현성 기자(hsyhs@newsis.com)
"""

In [43]:
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    """다음 뉴스 기사에서 관련 정보를 출력 스키마에 맞춰서 추출해주세요. 예시를 참고하세요.

    ## 뉴스 기사
    {news_content}

    ## 출력 스키마
    - title: 기사 제목
    - content: 기사 내용 (원문 그대로 추출)
    - press: 언론사
    - author: 작성자(예: 길동,홍 = '홍길동'이라는 이름의 given name, surname 순서로 기재)
    - published_date: 작성일(예: 2023년 1월 1일) 
    """
    )
"""
원래 교안에는 아래 내용 작성되어있었는데, Pydantic 클래스로 출력 구조를 정의한 것과 중복이라 생략
    ## 출력 스키마
    - title: 기사 제목
    - content: 기사 내용 (원문 그대로 추출)
    - press: 언론사
    - author: 작성자(예: 길동,홍)
    - published_date: 작성일(예: 2023년 1월 1일) 
"""

# 모델 생성 및 구조화된 출력 바인딩
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
structured_model = model.with_structured_output(NewsInfo)

# 프롬프트와 모델 체인 연결
chain = prompt | structured_model

# 체인 실행
result = chain.invoke({"news_content": news_article})

In [44]:
# 결과 출력 (NewsInfo 객체)
print(type(result))
print("-" * 20)
print(result)
print("-" * 20)
print(f"기사 제목: {result.title}")
print(f"언론사: {result.press}")
print(f"작성자: {result.author}")
print(f"작성일: {result.published_date}")

<class '__main__.NewsInfo'>
--------------------
title='"복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발' content='입력2025.06.20. 오후 12:00  수정2025.06.20. 오후 1:39 \n\n[서울=뉴시스]윤현성 기자 = 정부가 공공분야에 도입할 초거대 인공지능(AI) 기반 국민 편의 서비스 본격 개발에 나선다. 세금 납부 시 생성형 AI 챗봇을 통해 어려운 세무 용어 등에 대한 설명·상담을 듣거나, 대량의 민원업무도 생성형 AI 기반 분석으로 빠르게 답변·응대하는 등의 형태다.\n\n과학기술정보통신부와 디지털플랫폼정부위원회는 2025년도 \'초거대 AI 서비스 개발 지원\' 사업을 본격 추진하기 위해 수행기업 공모를 실시한다고 20일 밝혔다. 이 사업은 공공분야에 초거대 AI를 도입·확산하고 이를 통해 행정 효율화, 대국민 서비스 혁신, 사회현안 해결이 가능한 서비스 개발을 목표로 추진된다.\n\n올해는 다양한 공공분야에서 초거대 AI 기술을 통해 국민이 체감할 수 있고 실질적인 변화를 가져올 수 있는 과제를 중점적으로 발굴했다. 중앙부처·지자체·공공기관을 대상으로 1~2월에 과제 공모를 추진했으며 총 5개 과제가 선정됐다.\n\n국민권익위원회의 \'생성형 AI 기반 국민소통·민원분석 체계 구축\'은 국민소통시스템에 생성형 AI 기반 민원분석 체계를 도입해 민원처리 행정 효율화와 답변품질을 향상시킨다. 이를 통해 대량의 민원업무를 신속·효율적으로 대응해 민원업무의 효율성을 증대하고 국민의 신뢰도 향상에도 기여할 것으로 기대된다.\n\n국세청의 \'생성형 AI 기반 국세 상담 지원 서비스\'는 납세자가 홈택스 이용 시 전자신고 관련한 문의사항을 즉시 해소할 수 있는 실시간 상담 서비스를 제공한다. 홈택스에 상담전용 AI챗봇을 도입해 전화 상담 시 발생하는 장시간 대기 문제를 해결하고, 어려운 세무 용어 등으로 인한 불편사항을 개선할 예정이다.\n\n산업통상자원부의 \'해외인증

In [45]:
# 결과를 JSON으로 출력
result.model_dump()

{'title': '"복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발',
 'content': '입력2025.06.20. 오후 12:00  수정2025.06.20. 오후 1:39 \n\n[서울=뉴시스]윤현성 기자 = 정부가 공공분야에 도입할 초거대 인공지능(AI) 기반 국민 편의 서비스 본격 개발에 나선다. 세금 납부 시 생성형 AI 챗봇을 통해 어려운 세무 용어 등에 대한 설명·상담을 듣거나, 대량의 민원업무도 생성형 AI 기반 분석으로 빠르게 답변·응대하는 등의 형태다.\n\n과학기술정보통신부와 디지털플랫폼정부위원회는 2025년도 \'초거대 AI 서비스 개발 지원\' 사업을 본격 추진하기 위해 수행기업 공모를 실시한다고 20일 밝혔다. 이 사업은 공공분야에 초거대 AI를 도입·확산하고 이를 통해 행정 효율화, 대국민 서비스 혁신, 사회현안 해결이 가능한 서비스 개발을 목표로 추진된다.\n\n올해는 다양한 공공분야에서 초거대 AI 기술을 통해 국민이 체감할 수 있고 실질적인 변화를 가져올 수 있는 과제를 중점적으로 발굴했다. 중앙부처·지자체·공공기관을 대상으로 1~2월에 과제 공모를 추진했으며 총 5개 과제가 선정됐다.\n\n국민권익위원회의 \'생성형 AI 기반 국민소통·민원분석 체계 구축\'은 국민소통시스템에 생성형 AI 기반 민원분석 체계를 도입해 민원처리 행정 효율화와 답변품질을 향상시킨다. 이를 통해 대량의 민원업무를 신속·효율적으로 대응해 민원업무의 효율성을 증대하고 국민의 신뢰도 향상에도 기여할 것으로 기대된다.\n\n국세청의 \'생성형 AI 기반 국세 상담 지원 서비스\'는 납세자가 홈택스 이용 시 전자신고 관련한 문의사항을 즉시 해소할 수 있는 실시간 상담 서비스를 제공한다. 홈택스에 상담전용 AI챗봇을 도입해 전화 상담 시 발생하는 장시간 대기 문제를 해결하고, 어려운 세무 용어 등으로 인한 불편사항을 개선할 예정이다.\n\n산업통상자원부의 \'해외인증 공공특화 AI 에이전트 서비스\'는 모바일 플랫폼, 소셜네트워크서비스 

---

### [실습] 

- 구조화된 출력을 사용하여, 뉴스 기사에서 언론사, 기사 제목, 기사 내용, 작성자, 작성일을 추출해보세요.

In [ ]:
# 실제 뉴스 기사를 복사하여 사용하세요.
# news_article은 위 Cell에 정의되어 있습니다.